In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
import cv2,os
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from keras.layers import Conv2D,MaxPooling2D,Dense,Activation,Dropout

In [3]:
datapath=r"E:\dphi\face_mask_detection\train"

In [4]:
categories=os.listdir(datapath)

In [5]:
print(categories)

['without_mask', 'with_mask']


In [6]:
labels=[i for i in range(len(categories))]

In [7]:
print(labels)

[0, 1]


In [8]:
label_dict=dict(zip(categories,labels))

In [9]:
print(label_dict)

{'without_mask': 0, 'with_mask': 1}


In [10]:
img_size=100
data=[]
target=[]
for category in categories:
    folder_path=os.path.join(datapath,category)
    img_path=os.listdir(folder_path)
    for img_names in img_path:
        imgp=os.path.join(folder_path,img_names)
        img=cv2.imread(imgp)        
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        resized=cv2.resize(gray,(img_size,img_size))
#         img=image.img_to_array(resized)
#         img1=np.expand_dims(resized,axis=0)
        data.append(resized)
        target.append(label_dict[category])

In [11]:
target=np.array(target)
target

array([0, 0, 0, ..., 1, 1, 1])

In [12]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
from keras.utils import np_utils
target=np_utils.to_categorical(target)

In [13]:
data.shape

(11264, 100, 100, 1)

In [49]:
from keras.layers import Flatten
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=data.shape[1:],activation="relu"))     
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))     
model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D(128,(3,3),activation="relu"))     
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Conv2D(256,(3,3),activation="relu"))     
# model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(.5))          
model.add(Dense(50,activation="relu"))
model.add(Dense(2,activation="softmax"))

In [50]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 98, 98, 32)        320       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 33856)             0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 33856)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 50)               

In [51]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [52]:
# from keras.optimizers import SGD
# sgd=SGD(lr=.01,decay=1e-6,momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [53]:
model.fit(train_data,train_target,validation_split=.2,epochs=5,batch_size=30)

Epoch 1/5
271/271 [==============================] - 63s 234ms/step - loss: 0.2883 - accuracy: 0.8843 - val_loss: 0.1918 - val_accuracy: 0.9349
Epoch 2/5
271/271 [==============================] - 68s 252ms/step - loss: 0.1620 - accuracy: 0.9462 - val_loss: 0.1618 - val_accuracy: 0.9463
Epoch 3/5
271/271 [==============================] - 66s 243ms/step - loss: 0.1335 - accuracy: 0.9540 - val_loss: 0.1512 - val_accuracy: 0.9532
Epoch 4/5
271/271 [==============================] - 67s 248ms/step - loss: 0.1075 - accuracy: 0.9628 - val_loss: 0.1383 - val_accuracy: 0.9571
Epoch 5/5
271/271 [==============================] - 67s 248ms/step - loss: 0.0858 - accuracy: 0.9686 - val_loss: 0.1426 - val_accuracy: 0.9522


In [16]:
#if you want to test on several images
# testingp=r"E:\dphi\face_mask_detection\test"
# images=os.listdir(testingp)

In [25]:
# from tensorflow.keras.preprocessing import image
# prd=[]
# for img in images:
#     path=os.path.join(testingp,img)
#     img=image.load_img(path,target_size=(100,100))
#     img=image.img_to_array(img)
#     img=np.expand_dims(img,axis=0)
#     pr=model.predict(img)
#     if pr[0][0]>=1:
#         prd.append("without_mask")
#     else:
#         prd.append("with_mask")

In [17]:

# for i in prd:
#     print(f"\'{i}\',")

In [58]:
#Live detection using Opencv
face_classifier=cv2.CascadeClassifier('C:/Users/Lenovo/AppData/Local/Programs/Python/Python38/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')
from tensorflow.keras.preprocessing import image
labels_dict={0:'NO MASK',1:'MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}
cap = cv2.VideoCapture(0)
while True:

    ret, frame = cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5) 
    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        normalized=np.expand_dims(normalized,axis=0)
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2)
    cv2.imshow("live",frame)
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()